# 数值最优化

In [ ]:
library(magrittr)

除了少数贝叶斯估计量，几乎所有计量经济学的估计量，例如OLS、MLE、2SLS和GMM，都是由目标函数求解最优化得出的。在计量经济学的学习中，许多人往往知其然不知其所以然，在使用计量方法时，只能依赖已经设定好的计量程序包来进行运算，完全无法按照自己模型的需求自行设定运算程序。甚至有时候，他们会调整自己的数据来迎合现成的计量方法，来产出看似“显著”的结果。背后一个重要的原因就是大部分的计量经济学课程中忽视了最优化问题的教学。如果想要不再被动地使用他人设定好的程序包，关键就在于学会如何构建并求解最优化问题。


一个常规的最优化函数可以写作：

$$
\min_{\theta \in \Theta } f(\theta) \,\, \mathrm{ s.t. }  g(\theta) = 0, h(\theta) \leq 0,
$$

其中 $f(\cdot)\in \mathbb{R}$ 是一个标量目标函数， $g(\theta) = 0$ 是等式约束条件，
而 $h(\theta)\leq 0$ 则是不等式约束。

目前的数值最优化算法都是在求解局部的最小值。有时候我们会遇到多个局部极小值同时存在的情况，此时我们无法保证能够找到全局的最小值。

不带有等式/不等式约束条件的最优化问题被称为**无约束**问题；相反则称之为**约束**问题。
约束条件可以通过拉格朗日乘数法包含在目标函数之中。拉格朗日乘数法对于学习经济学的学生来说并不陌生，在微观经济学当中我们就经常遇到。在学习消费者行为的时候，我们经常需要求解给定柯布-道格拉斯效用函数和预算约束的效用最大化的问题，此时我们往往就是运用拉格朗日乘数法再求偏导得到。

在实际运用中，我们总是会遇到简洁和效率的取舍问题。在这里简洁指的是数学表达式和代码的可读性，而效率则是考虑电脑运算的速度。通常建议在前期的试错和调试阶段更侧重于简洁性来节省时间和精力；在比较成熟的后期阶段进行全面的效率的提升。

## 方法

在运筹学中有许多的最优化算法。它们在一系列基本原理的基础上发展出来的各种变体。许多标准的MLE估计量都是二阶可微的，但并不存在显性解，比如Logit，Probit和Tobit。二阶可微目标函数最优化的核心思路是牛顿法。

最优化问题中首要的必要条件是函数的一阶条件：
$s(\theta) = \partial f(\theta) / \partial \theta = 0$.

In [ ]:
f <- function(x) 0.1 * (x - 5)^2 + cos(x) # criterion
s <- function(x) 0.2 * (x - 5) - sin(x) # gradient
h <- function(x) 0.2 - cos(x) # Hessian

# plot
par(mfrow = c(3, 1))
par(mar = c(2, 4, 1, 2))

x_base <- seq(0.1, 10, by = 0.1)
plot(y = f(x_base), x = x_base, type = "l", lwd = 2, ylab = "f")
plot(y = s(x_base), x = x_base, type = "l", ylab = "score")
abline(h = 0, lty = 2)
plot(y = h(x_base), x = x_base, type = "l", ylab = "Hessian")
abline(h = 0, lty = 2)


设定初始值 $\theta_0$ ，如果 $s(\theta_0) \neq 0$ ，那么对于 $t=0,1,\cdots$ ，进行以下迭代：

$$
\theta_{t+1} = \theta_{t} -  \left( H(\theta_t)  \right)^{-1}  s(\theta_t)
$$

其中
$H(\theta) = \frac{ \partial s(\theta )}{ \partial \theta}$ 是黑塞矩阵
(Hessian matrix)。这个公式不难推导，以 $\theta_t$ 为中心点进行泰勒展开就能够直观地得到。由于 $\theta_{ \star }$ 是 $s(\cdot)$ 的一个根，可以写作：

$$
0 = s(\theta_{\star}) = s(\theta_t) + H(\theta_t) (\theta_{t+1} - \theta_t) + O( (\theta_{t+1} - \theta_t)^2 ).
$$

忽略高阶项重新整理得到
$\theta_{\star} = \theta_{t} -  \left( H(\theta_t)  \right)^{-1}  s(\theta_t),$ 
之后用新的 $\theta_{t+1}$ 替代 $\theta_{\star}$ 就能写出迭代公式。换言之，这是非线性函数 $s(\cdot)$ 的一阶线性迭代公式。运算一直迭代到
 $|\theta_{t+1} -\theta_{t}| < \epsilon$ (绝对标准) 且/或
$|\theta_{t+1} -\theta_{t}|/|\theta_{t}| < \epsilon$ (相对标准)才会停止，其中
$\epsilon$ 是人为选定的宽容度(通常为较小的正数)。


In [ ]:
# Newton's method
Newton <- function(x) {
  x - s(x) / h(x)
} # update formula

x_init <- 6 # can experiment with various initial values

gap <- 1
epsilon <- 0.0001 # tolerance
while (gap > epsilon) {
  x_new <- Newton(x_init) %>% print()
  gap <- abs(x_init - x_new)
  x_init <- x_new
}

**牛顿法** 牛顿法的主要思路是求出函数 $s(\theta) = 0$ 的解。前文提到，一阶条件是最优化的必要不充分条件。我们需要验证每一个解 $s(\theta)$ 的二阶条件来决定它是极小值、极大值还是鞍点。如果有多个极小值，我们还需要比较它们之间的大小来决定哪一个是全局的最小值。

显然，牛顿法需要计算梯度 $s(\theta)$ 和黑塞矩阵 $H(\theta)$ 。
牛顿法从数值上收敛于二次率。


**拟牛顿法**(Quasi-Newton Method) 最知名的拟牛顿算法是  
[BFGS](http://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm)。
它回避了计算量巨大的Hessian matrix的直接运算，而是从一个初始的可逆Hessian开始，用一个直接的公式迭代出最终的矩阵。这个公式可以用二次近似的思想推导出来。

**无梯度法** [Nelder-Mead](http://en.wikipedia.org/wiki/Nelder%E2%80%93Mead_method)
是一个简单的方法。它通过映射和放缩来寻找局部的最小值。 

## 实施

R 语言的最优化程序一直在完善和提升中。
[R Optimization Task View](http://cran.r-project.org/web/views/Optimization.html)
概述了所有可用的CRAN拓展包。为了处理普通的非线性最优化问题，程序包
[`optimx`](http://cran.r-project.org/web/packages/optimx/index.html) [{cite}`nash2014best`]
替换了R语言默认的最优化指令。 `optimx` 为数种常用的最优化算法提供了一个统一的操作界面。此外，它还能够比较不同算法的计算结果。
[`ROI`](https://cran.r-project.org/web/packages/ROI/index.html) [{cite}`epubwu5858`] 是一个相对较新的程序包，它想要提供一个研究者能够相互交流的统一的模型框架。未来我们将把`ROI`的介绍纳入到我们的文稿中。

**实例**

我们使用 `optimx` 来求解伪泊松极大似然估计(pseudo Poisson maximum likelihood estimation PPML)。这种方法在国际贸易的跨国双边贸易中经常用到。 [{cite}`silva2006log`]. 显然，如果 $y_i$ 是连续随机变量，它就不服从泊松分布，因为泊松分布是离散的非负分布。但如果满足条件均值模型

$$
E[y_i | x_i] = \exp( x_i' \beta)
$$ 

我们依然可以用泊松分布来获得参数 $\beta$ 的相合估计量。也就是说，哪怕 $y_i$ 不遵循条件泊松分布，依然不影响该方法成立。

如果 $Z$ 服从均值为 $\lambda$ 的泊松分布，概率质量函数为：

$$
\Pr(Z = k) = \frac{\mathrm{e}^{-\lambda} \lambda^k}{k!}, \mathrm{ for }\, \, k=0,1,2,\ldots,
$$

从而

$$
\log \Pr(Y = y | x) =  -\exp(x'\beta) + y\cdot x'\beta - \log k!
$$

又因为最后一项与参数无关，所以对数似然函数为：

$$
\ell(\beta) = \log \Pr( \mathbf{y} | \mathbf{x};\beta ) =
-\sum_{i=1}^n \exp(x_i'\beta) + \sum_{i=1}^n y_i x_i'\beta.
$$

此外，我们能够轻松地写出梯度

$$
s(\beta) =\frac{\partial \ell(\beta)}{\partial \beta} =
-\sum_{i=1}^n \exp(x_i'\beta)x_i + \sum_{i=1}^n y_i x_i.
$$

并且验证Hessian矩阵为负定：

$$
H(\beta) = \frac{\partial^2 \ell(\beta)}{\partial \beta \partial \beta'} =
-\sum_{i=1}^n \exp(x_i'\beta)x_i x_i'
$$

因此 $\ell(\beta)$ 是 $\beta$ 的严格凹函数。

尽管效用和似然估计都是在求解最大值，但是运筹学领域通常默认最优化问题是在求解最小值。为了遵循运筹学中这一“传统”，这里我们提出 **负** 对数似然估计。

In [ ]:
# Poisson likelihood
poisson.loglik <- function(b) {
  b <- as.matrix(b)
  lambda <- exp(X %*% b)
  ell <- -sum(-lambda + y * log(lambda))
  return(ell)
}

如果要在 `R` 中处理最优化问题，最好将最优化的目标写成参数的函数。数据在函数内或者函数外都可以提供。如果数据使用arguments形式提供的，arguments必须是显性的。(相反，如果要在`Matlab`中进行最优化，那么参数必须是函数唯一的argument。)

In [ ]:
# implement both BFGS and Nelder-Mead for comparison.

library(AER)

## prepare the data
data("RecreationDemand")
y <- RecreationDemand$trips
X <- with(RecreationDemand, cbind(1, income))

## estimation
b.init <- c(0, 1) # initial value
b.hat <- optimx::optimx(b.init, poisson.loglik,
  method = c("BFGS", "Nelder-Mead"),
  control = list(
    reltol = 1e-7,
    abstol = 1e-7
  )
)
print(b.hat)


给定条件均值模型，非线性最小二乘(NLS)理论上也是相合的。
NLS最小化的目标函数为

$$
\sum_{i=1}^n (y_i - \exp(x_i \beta))^2
$$

我们不禁会想到一个问题：为什么我们会更青睐PPML而非NLS？ 

原因是PPML对于线性参数的最优化是凸优化，而NLS并不是(凸优化的定义和性质将在后面详细介绍)。

现实中，没有一个算法足以应对所有的优化问题。当真实参数已知时，模拟有助于让我们在实际应用之前检测最优化方法的准确性。等高线图(Contour plot)是一个在低维空间中对函数曲面/流形进行可视化的可靠工具。

**实例**

In [ ]:
x.grid <- seq(0, 1.8, 0.02)
x.length <- length(x.grid)
y.grid <- seq(-.5, .2, 0.01)
y.length <- length(y.grid)

z.contour <- matrix(0, nrow = x.length, ncol = y.length)

for (i in 1:x.length) {
  for (j in 1:y.length) {
    z.contour[i, j] <- poisson.loglik(c(x.grid[i], y.grid[j]))
  }
}

contour(x.grid, y.grid, z.contour, 20)


如果有一些问题对于准确性有更高的要求，我们可以在`R`中调用第三方的独立程序包来处理。比如我们可以用程序包[`nloptr`](http://cran.r-project.org/web/packages/nloptr/index.html)来调用[`NLopt`](http://ab-initio.mit.edu/wiki/index.php/NLopt_Installation)。


`NLopt` 提供了一系列的拓展算法(http://ab-initio.mit.edu/wiki/index.php/NLopt_Algorithms#SLSQP).

**实例**

我们可以先在NLOPT中运行Nelder-Mead算法：

In [ ]:
## optimization with NLoptr

opts <- list(
  "algorithm" = "NLOPT_LN_NELDERMEAD",
  "xtol_rel" = 1.0e-7,
  maxeval = 500
)

res_NM <- nloptr::nloptr(
  x0 = b.init,
  eval_f = poisson.loglik,
  opts = opts
)
print(res_NM)

# "SLSQP" is indeed the BFGS algorithm in NLopt,
# though "BFGS" doesn't appear in the name
opts <- list("algorithm" = "NLOPT_LD_SLSQP", "xtol_rel" = 1.0e-7)

要在NLOPT中调用BFGS，我们需要在下方的函数`poisson.log.grad()`中编写 $s(\beta)$ ：

In [ ]:
poisson.loglik.grad <- function(b) {
  b <- as.matrix(b)
  lambda <- exp(X %*% b)
  ell <- -colSums(-as.vector(lambda) * X + y * X)
  return(ell)
}


我们比较解析梯度与数值梯度来验证函数的正确性。

In [ ]:
# check the numerical gradient and the analytical gradient
b <- c(0, .5)
numDeriv::grad(poisson.loglik, b)
poisson.loglik.grad(b)

编写好梯度函数后，就可以开始运行BFGS。

In [ ]:
res_BFGS <- nloptr::nloptr(
  x0 = b.init,
  eval_f = poisson.loglik,
  eval_grad_f = poisson.loglik.grad,
  opts = opts
)
print(res_BFGS)

## 凸优化

**凸优化** 是一种较为特别的优化，它指的是最优化过程中需要求解的目标函数为凸函数。凸优化的优点在于它是所有的优化问题中最容易解决的一种。其中的一个重要原因在于凸优化中的局部最优解就是全局最优解，无需再考虑求解出多个局部最优的比较问题；另一个原因是凸函数本身的形式更易于求解(可以参考微观经济学效用最大化的计算问题)。因此，大多数我们遇到的非凸优化问题也是通过近似凸优化求解来处理的。

读者如果想要进一步学习凸优化可以参考文章{cite}`boyd2004convex`。当中有一句话特别有意思：“凸优化是技术，而其他的优化是艺术。”某种意义上来说，这句话是成立的。

In [ ]:
f1 <- function(x) x^2
f2 <- function(x) abs(x)
f3 <- function(x) (-x - 1) * (x <= -1) + (0.4 * x - .2) * (x >= .5)

par(mfrow = c(1, 3))
par(mar = c(4, 2, 1, 2))

x_base <- seq(-3, 3, by = 0.1)
plot(y = f1(x_base), x = x_base, type = "l", lwd = 2, xlab = "differentiable")
plot(y = f2(x_base), x = x_base, type = "l", lwd = 2, xlab = "non-differentiable")
plot(y = f3(x_base), x = x_base, type = "l", lwd = 2, xlab = "multiple minimizers")

**实例**

* 线性回归模型 MLE


Normal MLE. The (negative) log-likelihood is

$$
\ell (\beta, \sigma) = \log \sigma + \frac{1}{2\sigma^2}\sum_{i=1}^n (y_i - x_i' \beta)^2
$$

This is not a convex problem, because $\log \sigma$ is concave. But if we re-parameterize the criterion function by $\gamma = 1/\sigma$ and $\alpha = \beta / \sigma$, then

$$
\ell (\alpha, \gamma) = -\log \gamma + \frac{1}{2}
\sum_{i=1}^n (\gamma y_i - x_i' \alpha)^2
$$

 in convex in $\alpha, \gamma$. Many MLE estimators in econometric textbooks are convex. 

鉴于高阶估计问题的重要性，{cite}`gao2018two`用R对两个计量问题进行了凸优化的求解。 `CVXR` [@fu2018cvxr] 是一个便捷的凸模型程序语言，同时还能支持付费优化包 `CLEPX`, `MOSEK`, `Gurubi` 和开源优化包 `ECOS` 与 `SDPT3`的运行。 尽管开源优化包有无需付费和能够在云计算中处理大量数据的优点，但付费优化包还是更加可靠。 `MOSEK` 为学者们提供免费的学术通行证且有不错的用户体验。
 [`Rmosek`](http://rmosek.r-forge.r-project.org/) 提供了一个 `R` 的交互界面来下载 `Mosek` (但是在安装`Rmosek`前需要先下载好`Rtools`)

**例子：宽松经验似然**



想象一个有真实参数 $\beta_0$ 的模型，它满足矩条件 $\mathrm{E}\left[  h\left(Z_i, \beta_0 \right)\right] = 0_m$ ，其中 $\left\{Z_i \right\}_{i=1}^n$ 是观测值， $\beta$
是我们想要求解的低维参数，而 $h$ 是一个 $\mathbb{R}^{m}$ 中的矩条件 
经验似然 (EL) [{cite}`owen1988empirical`] [{cite}`qin1994empirical`] 计算

$$
\max_{\beta \in \mathcal{B}, \pi \Delta_n} \; \sum_{i=1}^n \log \pi_i \;\,\, \text{s.t.} \; \sum_{i=1}^n \pi_i h \left( Z_i, \beta \right) = 0_m
$$

其中 $\Delta_{n} = \left\{ \pi\in\left[0,1\right]^{n}:\sum_{i=1}^{n}\pi_{i}=1 \right\}$ 是 $n$ 维的概率单形体。

为了处理高维问题(比如 $m > n$ )， {cite}`shi2016econometric`提出了宽松经验似然(REL)，它被定义为如下式子的解：

$$
\max_{\beta\in\mathcal{B}}\max_{\pi\in\Delta_{n}^{\lambda}\left(\beta\right)}\,\sum_{i=1}^{n}\log\pi_{i}
$$

其中 

$$
\Delta_{n}^{\lambda}\left(\beta\right)=\left\{ \pi_i \in\Delta_{n}:\big|\sum_{i=1}^{n}\pi_{i}h_{ij}\left(\beta\right)\big|\leq\lambda,\:j=1,2,\cdots,m\right\}
$$

是一个宽松单形体， $\lambda\geq0$ 是一个调谐参数， $h_{ij}\left(\beta\right)=h_{j}\left(Z_{i},\beta\right)$
是 $h\left(Z_{i},\beta\right)$ 的第 $j$ 个元素。

与标准的EL类似， REL 的优化使用了内循环(inner loop)和外循环(outer loop)。
 $\beta$ 的外循环是普通的低阶非线性最优化，可以通过牛顿类方法求解。有线性约束和对数目标的内循环在 $\pi=\left(\pi_{i}\right)_{i=1}^{n}$ 中是外凸的。通过引入辅助变量 $t_i$ ，对数目标可以用线性目标函数$\sum_{i=1}^n t_i$ 和指数圆锥约束 $n$ 来表示： $\left(\pi_{i}, 1, t_{i}\right) \in \mathcal{K}_{\mathrm{exp}}=\left\{\left(x_{1}, x_{2}, x_{3}\right): x_{1} \geq x_{2} \exp \left(x_{3} / x_{2}\right), x_{2}>0\right\} \cup\left\{\left(x_{1}, 0, x_{3}\right): x_{1} \geq 0, x_{3} \leq 0\right\}$, $i=1,2,\cdots,n$ 。 



对于每个 $\beta$ 来说，内循环问题可以写成圆锥规划问题：

$$
\begin{aligned}
&\max _{\pi, t} \sum_{i=1}^{n} t_{i}\\
\text { s.t. }&\left[\begin{array}{c}
1 \\
-\lambda \\
\vdots \\
-\lambda
\end{array}\right] \leq\left[\begin{array}{cccc}
1 & 1 & \cdots & 1 \\
h_{11}(\beta) & h_{21}(\beta) & \cdots & h_{n 1}(\beta) \\
\vdots & \vdots & \ddots & \vdots \\
h_{1 m}(\beta) & h_{2 m}(\beta) & \cdots & h_{n m}(\beta)
\end{array}\right]\left[\begin{array}{c}
\pi_{1} \\
\pi_{2} \\
\vdots \\
\pi_{n}
\end{array}\right] \leq\left[\begin{array}{c}
1 \\
\lambda \\
\vdots \\
\lambda
\end{array}\right]\\
&\left(\pi_{i}, 1, t_{i}\right) \in \mathcal{K}_{\mathrm{exp}}, 0 \leq \pi_{i} \leq 1, \text { for each } i=1,2, \cdots, n
\end{aligned}
$$

我们可以这样理解指数圆锥：注意到 
$\left(\pi_{i}, 1, t_{i}\right) \in \mathcal{K}_{\mathrm{exp}}$ 与
$\{ \pi_i \geq \exp(t_i): \pi_i\geq 0, t_i \leq 0 \}$ 相等，因此 
$t_i \leq \log \pi_i$ 。由于我们需要最大化 $\sum t_i$ ，因此
$t_i = \log \pi_i$ 必须成立。约束最优化在 `Rmosek` 中可以运行，它可以将数学表达式转换为编程语言。



In [ ]:
innerloop <- function(b, y, X, Z, tau) {
  n <- nrow(Z)
  m <- ncol(Z)

  # Generate moment condition
  H <- MomentMatrix(y, X, Z, b)

  # Initialize the mosek problem
  Prob <- list(sense = "max")

  # Prob$dparam$intpnt_nl_tol_rel_gap <- 1e-5;
  Prob$dparam <- list(INTPNT_CO_TOL_REL_GAP = 1e-5)

  # Linear coefficients of the objective
  Prob$c <- c(rep(0, n), rep(1, n), rep(0, n))

  # Linear constraints
  H_tilde <- Matrix(rbind(rep(1, n), H), sparse = TRUE)
  A <-
    rbind(
      cbind(H_tilde, Matrix(0, m + 1, 2 * n, sparse = TRUE)),
      cbind(Matrix(0, n, 2 * n, sparse = TRUE), Diagonal(n))
    )
  Prob$A <- A
  Prob$bc <-
    rbind(c(1, rep(-tau, m), rep(1, n)), c(1, rep(tau, m), rep(1, n)))
  Prob$bx <- rbind(
    c(rep(0, n), rep(-Inf, n), rep(1, n)),
    c(rep(1, n), rep(0, n), rep(1, n))
  )

  # Exponential Cones
  NUMCONES <- n
  Prob$cones <- matrix(list(), nrow = 2, ncol = NUMCONES)
  rownames(Prob$cones) <- c("type", "sub")
  for (i in 1:n) {
    Prob$cones[, i] <- list("PEXP", c(i, 2 * n + i, n + i))
  }

  # Invoke Mosek
  mosek.out <- mosek(Prob, opts = list(verbose = 0, soldetail = 1))

  if (mosek.out$sol$itr$solsta == "OPTIMAL") {
    # Since the default of NLOPTR is to do minimization, need to set it as negative
    return(-mosek.out$sol$itr$pobjval)
  } else {
    warning("WARNING: Inner loop not optimized")
    return(Inf)
  }
}


内循环最优化也可以用 `CVXR` 运算。它的程序更加简短易懂。

In [ ]:
innerloop.cvxr <- function(b, y = NULL, X = NULL, Z = NULL, tau = NULL) {
  n <- nrow(Z)
  m <- ncol(Z)

  H <- MomentMatrix(y, X, Z, b)

  p <- Variable(n)
  constr <- list(
    sum(p) == 1,
    p >= 0,
    p <= 1,
    H %*% p >= -tau,
    H %*% p <= tau
  )

  obj <- sum(log(p))
  obj <- Maximize(obj)

  Prob <- Problem(obj, constr)
  cvxr.out <- solve(Prob)

  if (cvxr.out$status == "optimal") {
    return(-cvxr.out$value)
  } else {
    warning("WARNING: Inner loop not optimized")
    return(Inf)
  }
}


## 后续写作安排

* 更多的最优化问题，比如 Lasso, portfolio optimization (Shi, Su, and Xie 2020)
* 添加 `ROI`.

## 拓展阅读

* {cite}`fu2018cvxr`
* {cite}`gao2018two`
* {cite}`epubwu5858`


## 参考文献
